In [2]:
import meep as mp
import numpy as np
import math
import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('TkAgg')

height=2
length_Air=2
length_SiO2=0.25*0.7
length_Si=2

length=length_Air+length_SiO2+length_Si

geometry_1 = [
    mp.Block(
        mp.Vector3(height, mp.inf, length_Air),
        center=mp.Vector3(0,0,-(length_Air/2+length_SiO2)),
        material=mp.Medium(epsilon=1),
    ),
    mp.Block(
        mp.Vector3(height, mp.inf, length_SiO2),
        center=mp.Vector3(0,0,-length_SiO2/2),
        material=mp.Medium(epsilon=2.1025),
    ),
    mp.Block(
        mp.Vector3(height, mp.inf, length_Si),
        center=mp.Vector3(0,0,length_Si/2),
        material=mp.Medium(epsilon=11.8),
    ),
]

pml_layers = [mp.PML(thickness=0.5,direction=mp.Z)]

resolution = 50

fmax=1/0.4
fmin=1/0.7
fcen=(fmax+fmin)/2
df=fmax-fmin

geometry_2=[
    mp.Block(
        mp.Vector3(height, mp.inf, length),
        center=mp.Vector3(0,0,0),
        material=mp.Medium(epsilon=1),
    ),
]

nfreq = 100

theta=np.arange(-90,100,10)

cell_2=mp.Vector3(0, 0, length)

reflectance_700nm=[]


theta_r = math.radians(theta[0])

k = mp.Vector3(math.sin(theta_r),0,math.cos(theta_r)).scale(fmin)

if theta_r == 0:
    dimensions = 1
else:
    dimensions = 3

sources_3 = [mp.Source(mp.GaussianSource(fcen,fwidth=df),
                    component=mp.Ex,
                    center=mp.Vector3(0, 0, -(length_Air+length_SiO2)+1))]

sim3 = mp.Simulation(cell_size=cell_2,
                    geometry=geometry_2,
                    boundary_layers=pml_layers,
                    sources=sources_3,
                    k_point=k,
                    dimensions=dimensions,
                    resolution=resolution)

refl_fr = mp.FluxRegion(center=mp.Vector3(0,0,-(length_Air+length_SiO2)+1.5))
refl = sim3.add_flux(fcen, df, nfreq, refl_fr)

sim3.run(until_after_sources=mp.stop_when_fields_decayed(50, mp.Ex, mp.Vector3(0,0,-(length_Air+length_SiO2)+1.5), 1e-9))



-----------
Initializing structure...
time for choose_chunkdivision = 4.1008e-05 s
Working in 3D dimensions.
Computational cell is 0.02 x 0.02 x 4.18 with resolution 50
     block, center = (0,0,0)
          size (2,1e+20,4.175)
          axes (1,0,0), (0,1,0), (0,0,1)
          dielectric constant epsilon diagonal = (1,1,1)
time for set_epsilon = 0.00235415 s
-----------
Meep: using complex fields.
field decay(t = 50.01): 0.11269813552884442 / 0.11269813552884442 = 1.0
field decay(t = 100.01): 3.3415551647575873e-12 / 0.11269813552884442 = 2.9650491989748455e-11
run 0 finished at t = 100.01 (10001 timesteps)


In [ ]:
plt.figure(dpi=100)
sim3.plot2D()